In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(81)

### Softmax Loss Function Implement(Low Level) ###

In [ ]:
z = torch.FloatTensor([1, 2, 3])

In [ ]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
print(hypothesis.sum())

In [ ]:
z = torch.rand(3, 5, requires_grad=True)
print(z)

In [ ]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)
print(hypothesis.sum())

In [ ]:
y = torch.randint(5, (3,)).long()
print(y)

In [ ]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

In [ ]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

### Softmax Loss Function Implement(High Level) ###

In [ ]:
# torch.log(F.softmax(z, dim=1)) == F.log_softmax(z, dim=1)
torch.log(F.softmax(z, dim=1))

In [ ]:
F.log_softmax(z, dim=1)

In [ ]:
# First
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

# Second
cost = (y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()
print(cost)

# Third
cost = F.nll_loss(F.log_softmax(z, dim=1), y)
print(cost)

# Fourth
cost = F.cross_entropy(z, y)
print(cost)

### EXAMPLE

In [ ]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

print(x_train.shape)
print(y_train.shape)

In [ ]:
# One-Hot-Encoding; y_train에서 클래스는 0, 1, 2 총 3개이므로 결과는 8x3 Tensor가 나와야 함

y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

In [ ]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1, 3), requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)
print(optimizer)

In [ ]:
# Low Level
nb_epochs = 1000
for epoch in range(nb_epochs+1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [ ]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1, 3), requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)
print(optimizer)

In [ ]:
# High Level
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [ ]:
# nn.Module

model = nn.Linear(4, 3)
optimizer = optim.SGD(model.parameters(), lr=0.1)
print(optimizer)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [ ]:
# Class Implement

class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)  # Output이 3

    def forward(self, x):
        return self.linear(x)

In [ ]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))